<a href="https://colab.research.google.com/github/lidorsandak/ad_ML_Competition_m5/blob/main/data_eda_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M5 Forecasting Challenge: Predicting Daily Walmart Revenue: EDA and Preprocessing

**Course:** Advanced Machine Learning - Final Assignment

---

In [53]:
# Cell 1: Setup and Data Loading
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# First, you might need to install Optuna
!pip install optuna -q

import optuna

# General imports
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from statsmodels.tsa.seasonal import seasonal_decompose

from math import ceil

from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

warnings.filterwarnings('ignore')

Load Main Data

In [ ]:
# Define path to your data on Google Drive
DATA_PATH = '/content/drive/MyDrive/M5_Project/'


train_df = pd.read_csv(DATA_PATH + 'train.csv')
calendar_df = pd.read_csv(DATA_PATH + 'calendar_events.csv')
sample_submission = pd.read_csv(DATA_PATH + 'forecast_submission.csv')

Explore train data

In [ ]:
print('  unique forecasts: %i' % train_df.shape[0])
for col in train_df.columns:
  print('   N_unique %s: %i' % (col, train_df[col].nunique()))

train_df.head()


In [ ]:
# Check for missing dates per store
def check_missing_dates(df):
    """Identify gaps in the time series for each store"""
    df['date'] = pd.to_datetime(df['date'])

    for store_id in df['store_id'].unique():
        store_data = df[df['store_id'] == store_id].sort_values('date')
        date_range = pd.date_range(start=store_data['date'].min(),
                                     end=store_data['date'].max(),
                                     freq='D')
        missing_dates = date_range.difference(store_data['date'])

        if len(missing_dates) > 0:
            print(f"Store {store_id}: {len(missing_dates)} missing dates")
            print(f"  First few: {missing_dates[:5].tolist()}")

check_missing_dates(train_df)

In [ ]:

summary_by_store = train_df.groupby('store_id').agg(
    min_date=('date', 'min'),
    max_date=('date', 'max'),
    min_revenue=('revenue', 'min'),
    max_revenue=('revenue', 'max'),
    avg_revenue=('revenue', 'mean') # It's also useful to see the average
).reset_index()

# Improve formatting for readability
summary_by_store['min_revenue'] = summary_by_store['min_revenue'].round(2)
summary_by_store['max_revenue'] = summary_by_store['max_revenue'].round(2)
summary_by_store['avg_revenue'] = summary_by_store['avg_revenue'].round(2)

# Display the resulting table
print("Summary of Data Range and Revenue by Store ID:")
display(summary_by_store)

In [ ]:
# Check for zero/negative revenues (potential data issues)
print("\nZero revenue counts by store:")
print(train_df[train_df['revenue'] == 0].groupby('store_id').size())

print("\nNegative revenue counts:")
print((train_df['revenue'] < 0).sum())

Explore submission dates

In [ ]:
# The submission file ID is store_id + date (YYYYMMDD)
# We need to parse the date part of the 'id' string.
submission_ids = sample_submission['id'].str.split('_', expand=True)
submission_ids.columns = ['store_id_str', 'date_str']
submission_dates = pd.to_datetime(submission_ids['date_str'], format='%Y%m%d')

H = submission_dates.nunique()
print(f"The Forecast Horizon (H) is: {H} days.")

# Now find the min and max dates
min_submission_date = submission_dates.min()
max_submission_date = submission_dates.max()

# Calculate the duration (our forecast horizon H)
num_forecast_days = (max_submission_date - min_submission_date).days + 1

print(f"Submission File Analysis:")
print(f"-------------------------")
print(f"Forecast Start Date: {min_submission_date.date()}")
print(f"Forecast End Date:   {max_submission_date.date()}")
print(f"Total Forecast Horizon (H): {num_forecast_days} days")

##### **Data Merging and Preparation**

In [ ]:
# Convert 'date' columns to datetime objects
train_df['date'] = pd.to_datetime(train_df['date'])
calendar_df['date'] = pd.to_datetime(calendar_df['date'])

# Merge train_df with calendar_df to get all features in one place
original_df = pd.merge(train_df, calendar_df, on='date', how='left')
original_df['event'].fillna('NoEvent', inplace=True)

print("\nData merged:")
original_df.head()


--------------


## EDA



*   Temporal Patterns Analysis
    *   Overall Trend
    *   Seasonality Detection
    *   Year-over-Year Growth
*   Store-Level Analysis
    * Revenue Distribution Across Stores
    * Store Time Series Comparison
* Stationarity & Autocorrelation
* Event/Anomaly Detection
* Correlation Analysis
* Decomposition Analysis



In [ ]:
# ------------------- Temporal Patterns Analysis ---------------------

# A. Overall Trend

# Plot aggregate revenue over time
df_agg = train_df[train_df['store_id'] == 0].copy()
df_agg = df_agg.sort_values('date')

# Plot 1: Aggregate Revenue (across the 10 real stores)
train_stores= train_df[train_df['store_id'] != 0].copy()
df_sum=train_stores.groupby('date')['revenue'].sum()
df_sum.plot(title='Total Revenue Across 10 Stores', linewidth=0.8, figsize=(15, 5))
plt.tight_layout()
plt.show()

# Plot 2: Weekly Seasonality

plt.figure(figsize=(15, 5))
plt.plot(df_agg['date'], df_agg['revenue'], linewidth=0.8)
plt.title('Total Revenue Over Time (All Stores)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Revenue')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# B. Seasonality Detection

# Extract temporal features
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day_of_week'] = train_df['date'].dt.dayofweek
train_df['quarter'] = train_df['date'].dt.quarter

# Monthly seasonality
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Boxplot by month
train_df[train_df['store_id'] == 0].boxplot(column='revenue', by='month', ax=axes[0])
axes[0].set_title('Revenue Distribution by Month')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Revenue')

# Average revenue by day of week
train_df[train_df['store_id'] == 0].groupby('day_of_week')['revenue'].mean().plot(kind='bar', ax=axes[1])
axes[1].set_title('Average Revenue by Day of Week')
axes[1].set_xlabel('Day of Week (0=Mon, 6=Sun)')
axes[1].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], rotation=0)
axes[1].set_ylabel('Average Revenue')

plt.tight_layout()
plt.show()

# C. Compare Same Time Periods

# Filter to only Jan-Sep for all years to make fair comparisons
df_total = train_df[train_df['store_id'] == 0].copy()
df_total['date'] = pd.to_datetime(df_total['date'])
df_total['year'] = df_total['date'].dt.year
df_total['month'] = df_total['date'].dt.month

# Only use Jan-Sep (months 1-9) for all years
df_comparable = df_total[df_total['month'] <= 9]

yearly_revenue = df_comparable.groupby('year')['revenue'].sum()

print("Year-over-Year Revenue (Jan-Sep only):")
print(yearly_revenue)
print("\nYoY Growth Rate:")
yoy_growth = yearly_revenue.pct_change() * 100
print(yoy_growth)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
yearly_revenue.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Revenue by Year (Jan-Sep Only)', fontsize=12)
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Revenue')
axes[0].set_xticklabels(yearly_revenue.index, rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# Growth rate
yoy_growth.dropna().plot(kind='bar', ax=axes[1], color=['red' if x < 0 else 'green' for x in yoy_growth.dropna()])
axes[1].set_title('YoY Growth Rate (Jan-Sep)', fontsize=12)
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Growth Rate (%)')
axes[1].axhline(0, color='black', linewidth=0.8, linestyle='--')
axes[1].set_xticklabels(yoy_growth.dropna().index, rotation=0)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

**Insights and Modeling Implication**

- Weekly Seasonality:
    - Include day_of_week and is_weekend as features
    - Consider a weekly seasonal component in time series models (e.g., seasonal_period=7 in SARIMA)
- Monthly Seasonality:
    - The business is not highly seasonal at the monthly level
    - Events/holidays matter more than the month itself: Q4 outliers suggest end-of-year shopping boost
    - For model we should handle the events outliers so they won't skew our model
- Variance and Outliers
    - heteroscedasticity - variance increases over time

Holiday drops must be handled:
- Flag them as outliers and exclude from training
- Create is_holiday feature with special handling
- Use a model robust to outliers (e.g., Huber loss)


Key Takeaways:

* Log transformation is critical here (np.log1p(revenue)) in order to stabilize the increasing variance and makes the model multiplicative rather than an additive model
* Consider exogenous holiday variables
* The trend is not perfectly linear - might benefit from polynomial features or tree-based models

| Priority | Feature | Why |
|---|---|---|
| 🔥 **CRITICAL** | `day_of_week` / `is_weekend` | 40% weekend lift - massive signal |
| 🔥 **CRITICAL** | `is_holiday` / `event` | Explains the near-zero revenue days |
| 🔥 **CRITICAL** | Log transformation | Handles increasing variance |
| ⭐ **Important** | Lag features (you have these) | Revenue is autocorrelated (weekly pattern) |
| ⭐ **Important** | Rolling averages (you have these) | Smooth out noise, capture trends |
| 💡 **Helpful** | `month` / `quarter` | Mild seasonal effect, Q4 boost |
| 💡 **Helpful** | Year-based trend features | Capture the non-linear growth |

Model Selection Guidance:
Given these patterns, good model choices are:

1. Tree-Based Models (XGBoost, LightGBM):
- ✅ Handle non-linear growth pattern
- ✅ Automatically learn interactions (weekend × holiday)
- ✅ Robust to outliers
- ✅ Can handle your lag/rolling features well

2. SARIMA / Prophet (if you want pure time series):
- ✅ SARIMA with seasonal_order=(0,0,0,7) for weekly pattern
- ✅ Prophet excellent for handling holidays/events
- ⚠️ May struggle with the non-linear growth trend

3. Hybrid Approach (Recommended):
- Use Prophet/SARIMA for trend + seasonality
- Use XGBoost for the residuals (event effects, store-specific patterns)

In [ ]:
# ------------------- Store-Level Analysis ---------------------

# A.  Revenue Distribution Across Stores
df_stores = train_df[train_df['store_id'] != 0].copy()

# Average revenue by store
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

store_avg = df_stores.groupby('store_id')['revenue'].mean().sort_values()
store_avg.plot(kind='barh', ax=axes[0])
axes[0].set_title('Average Revenue by Store')
axes[0].set_xlabel('Average Revenue')

# Revenue variability by store (coefficient of variation)
store_cv = (df_stores.groupby('store_id')['revenue'].std() /
            df_stores.groupby('store_id')['revenue'].mean()).sort_values()
store_cv.plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Revenue Variability by Store (Coefficient of Variation)')
axes[1].set_xlabel('CV (lower = more stable)')

plt.tight_layout()
plt.show()

# B. Store Time Series Comparison
# Plot individual store trends
fig, axes = plt.subplots(5, 2, figsize=(15, 15))
axes = axes.flatten()

for i, store_id in enumerate(range(1, 11)):
    store_data = train_df[train_df['store_id'] == store_id].sort_values('date')
    axes[i].plot(store_data['date'], store_data['revenue'], linewidth=0.6)
    axes[i].set_title(f'Store {store_id}', fontsize=10)
    axes[i].tick_params(labelsize=8)
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

*Revenue Scale*

**High Revenue Tier:**

Store 3: ~$41K avg

Store 1: ~$29K avg

Store 6: ~$26K avg

**Middle Tier:**

Stores 7, 9, 10, 2, 5: ~$20-22K avg

**Low Revenue Tier**

Store 8: ~$18K avg

Store 4: ~$15K avg


Modeling Implication:
- Separate models per store (recommended, given clear tiers)
- Store ID as a categorical feature with embeddings (if using neural networks)
- Hierarchical/grouped forecasting (forecast total, then allocate to stores)


*Revenue Volatility: Stability vs. Unpredictability*

Most Stable (Easiest to Forecast):

- Store 6: CV ≈ 0.21 (🎯 most predictable)
- Store 4: CV ≈ 0.23
- Store 3: CV ≈ 0.24

Most Volatile (Hardest to Forecast):

- Store 9: CV ≈ 0.41 (⚠️ highly erratic)
- Store 8: CV ≈ 0.40
- Store 2: CV ≈ 0.28

Store 3 (highest revenue) is relatively stable, while Store 4 (lowest revenue) is also stable. Meanwhile, mid-tier stores like 8 and 9 are highly volatile.

Modeling Implication:
- Separate error bounds for each store, Store 9 will need wider prediction intervals
- More complex features needed for volatile stores (lag features, external data like weather/events)
- Simpler models may work for stable stores (Store 3, 4, 6)
Consider quantile regression for stores with high CV to capture uncertainty


Key Takeaways:

- Store 8

"# Create a regime indicator for Store 8
df.loc[(df['store_id'] == 8) & (df['date'] < '2012-06-01'), 'store_8_regime'] = 0
df.loc[(df['store_id'] == 8) & (df['date'] >= '2012-06-01'), 'store_8_regime'] = 1"

- Consider Store Clustering
  
  Instead of 10 separate models, group similar stores:
  
  Cluster A (High Growth): Stores 1, 3, 7

      Use models with strong trend components

  Cluster B (Stable/Mature): Stores 4, 6, 10

      Emphasize seasonality over trend

  Cluster C (Volatile): Stores 2, 8, 9

      Use robust models, more features, wider prediction intervals

  Cluster D (Middle): Store 5

      Standard approach

In [ ]:
# ------------------- Stationarity & Autocorrelation ---------------------


# Test for stationarity (Augmented Dickey-Fuller test)
def test_stationarity(timeseries, title):
    result = adfuller(timeseries.dropna())
    print(f'\n{title}')
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'  {key}: {value:.3f}')

    if result[1] <= 0.05:
        print("✓ Series is stationary (reject null hypothesis)")
    else:
        print("✗ Series is non-stationary (fail to reject null hypothesis)")

# Test on aggregate revenue
df_total = train_df[train_df['store_id'] == 0].sort_values('date').set_index('date')
test_stationarity(df_total['revenue'], 'Total Revenue Stationarity Test')

# ACF/PACF plots
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
plot_acf(df_total['revenue'].dropna(), lags=50, ax=axes[0])
plot_pacf(df_total['revenue'].dropna(), lags=50, ax=axes[1])
axes[0].set_title('Autocorrelation Function (ACF)')
axes[1].set_title('Partial Autocorrelation Function (PACF)')
plt.tight_layout()
plt.show()


In [ ]:
# ------------------- Event/Anomaly Detection ---------------------
# Detect outliers using IQR method
def detect_outliers(train_df, store_id):
    store_data = train_df[train_df['store_id'] == store_id].copy()
    Q1 = store_data['revenue'].quantile(0.25)
    Q3 = store_data['revenue'].quantile(0.75)
    IQR = Q3 - Q1

    outliers = store_data[
        (store_data['revenue'] < Q1 - 1.5 * IQR) |
        (store_data['revenue'] > Q3 + 1.5 * IQR)
    ]

    return outliers[['date', 'revenue']].sort_values('date')

# Check outliers for total revenue
print("Potential outlier dates (All Stores):")
outliers = detect_outliers(train_df, 0)
print(outliers.head(10))

# Visualize outliers
df_total = train_df[train_df['store_id'] == 0].sort_values('date')
Q1 = df_total['revenue'].quantile(0.25)
Q3 = df_total['revenue'].quantile(0.75)
IQR = Q3 - Q1

plt.figure(figsize=(15, 5))
plt.plot(df_total['date'], df_total['revenue'], linewidth=0.8, label='Revenue')
plt.axhline(Q3 + 1.5*IQR, color='r', linestyle='--', label='Upper Outlier Threshold')
plt.axhline(Q1 - 1.5*IQR, color='r', linestyle='--', label='Lower Outlier Threshold')
plt.title('Revenue with Outlier Thresholds')
plt.legend()
plt.tight_layout()
plt.show()



Consider corrected Outlier Detection Strategy

Event/Anomaly Insights
✅ Confirmed Patterns:

- Christmas Day = near-zero revenue every year (100% predictable)
- Multiple major holidays cause 99% revenue drops (need explicit handling)
- 2015 has unusual high-revenue days (requires investigation)
- IQR method flags 2015 growth as outliers (need year-specific thresholds)


🚨 Critical Actions:

- Add holiday features (is_christmas, is_major_holiday)
- Investigate 2015 high outliers - check event column and day_of_week
- Don't remove outliers blindly - holidays are predictable, not anomalies
- Consider separate models for holiday vs. non-holiday days

"# Option 1: Explicit features (best for tree models)
df['is_christmas'] = (df['date'].dt.month == 12) & (df['date'].dt.day == 25)
df['is_christmas'] = df['is_christmas'].astype(int)

"# Option 2: Exclude from training (for some models)
df_train = df[df['revenue'] > 1000]  # Remove near-zero days

"# Option 3: Use a library like 'holidays'
import holidays
us_holidays = holidays.US()
df['is_holiday'] = df['date'].apply(lambda x: x in us_holidays).astype(int)

In [ ]:


# ------------------- Correlation Analysis ---------------------

# If you have the 'event' column
if 'event' in train_df.columns:
    # Revenue by event type
    event_revenue = train_df[train_df['store_id'] == 0].groupby('event')['revenue'].agg(['mean', 'median', 'count'])
    print("\nRevenue by Event Type:")
    print(event_revenue.sort_values('mean', ascending=False))

# Weekend vs Weekday
train_df['is_weekend'] = train_df['day_of_week'].isin([5, 6]).astype(int)
weekend_comparison = train_df[train_df['store_id'] == 0].groupby('is_weekend')['revenue'].mean()
print(f"\nWeekday avg: ${weekend_comparison[0]:,.2f}")
print(f"Weekend avg: ${weekend_comparison[1]:,.2f}")
print(f"Weekend lift: {(weekend_comparison[1]/weekend_comparison[0] - 1)*100:.1f}%")

In [ ]:
# ------------------- Decomposition Analysis ---------------------



# Seasonal decomposition
df_total = train_df[train_df['store_id'] == 0].sort_values('date').set_index('date')

decomposition = seasonal_decompose(df_total['revenue'], model='additive', period=7)

fig, axes = plt.subplots(4, 1, figsize=(15, 10))
decomposition.observed.plot(ax=axes[0], title='Observed')
decomposition.trend.plot(ax=axes[1], title='Trend')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
decomposition.resid.plot(ax=axes[3], title='Residual')
plt.tight_layout()
plt.show()

In [ ]:
# Let's visualize the revenue distribution
sns.histplot(original_df['revenue'], kde=True, bins=50)
plt.title('Distribution of Revenue (Original)')
plt.show()

# Now apply log1p and visualize again
sns.histplot(np.log1p(original_df['revenue']), kde=True, bins=50)
plt.title('Distribution of Revenue (Log-Transformed)')
plt.show()

3. Feature Engineering



In [ ]:
def create_features(data):
    """
    Creates time series features from a datetime index.
    """
    # Make a copy to avoid modifying the original dataframe
    new_df = data.copy()

    # --- 1. Date-based Features ---
    # These help the model learn seasonal patterns
    new_df['date'] = pd.to_datetime(new_df['date'])
    new_df['day_of_week'] = new_df['date'].dt.dayofweek  # Monday=0, Sunday=6
    new_df['day_of_month'] = new_df['date'].dt.day
    new_df['week_of_year'] = new_df['date'].dt.isocalendar().week.astype(int)
    new_df['month'] = new_df['date'].dt.month
    new_df['year'] = new_df['date'].dt.year
    new_df['is_weekend'] = (new_df['day_of_week'] >= 5).astype(int) # 1 for weekend, 0 for weekday

    # --- 2. Lag Features ---
    h_forecast = H
    lags = [h_forecast, h_forecast + 7, h_forecast + 14, h_forecast + 28]

    # Sort data to ensure lags are correct
    new_df = new_df.sort_values(['store_id', 'date'])

    for lag in lags:
        new_df[f'revenue_lag_{lag}'] = new_df.groupby('store_id')['revenue'].shift(lag)

    # --- 3. Rolling Window Features ---
    # These capture recent trends and momentum.
    # We use a window that starts after the lag period.
    window_sizes = [7, 14, 28,46]
    for window in window_sizes:
        # The shift of h_forecast is crucial to prevent data leakage
        rolling_series = new_df.groupby('store_id')['revenue'].shift(h_forecast).rolling(window)
        new_df[f'revenue_roll_mean_{window}'] = rolling_series.mean()
        new_df[f'revenue_roll_std_{window}'] = rolling_series.std()

    # --- 4. Event Features ---
    # Convert text-based events into a format the model can use.
    # We will use one-hot encoding later, but for now we can make it a categorical type.
    new_df['event'] = new_df['event'].astype('category')

    # Convert store_id to category as well, so the model treats it as a label, not a number.
    new_df['store_id'] = new_df['store_id'].astype('category')

    return new_df

In [ ]:
# --- Apply the feature engineering function ---
#  Do we need log-transform to the revenue? original_df['revenue'] = np.log1p(original_df['revenue'])
df_featured = create_features(original_df)
df_featured['is_event'] = (df_featured['event'] != 'NoEvent').astype(int)
# Drop rows with NaN values created by lags and rolling windows
df_featured.dropna(inplace=True)
print(df_featured.head())
print(df_featured.columns)


In [ ]:
# Check if these are specific days of the week
df_store=df_featured[df_featured['store_id'] == 0]
high_outliers = df_store[df_store['date'].isin(['2015-01-31', '2015-02-07', '2015-02-08',
                                     '2015-03-08', '2015-03-15', '2015-04-12'])]
print(high_outliers[['date', 'revenue','store_id' ,'day_of_week', 'event']])

# Check if this is a data error - are they all the same day pattern?
print(high_outliers['day_of_week'].value_counts())

# Compare to normal 2015 Saturdays
normal_saturdays_2015 = df_store[(df_store['year'] == 2015) & (df_store['day_of_week'] == 5) &
                            (~df_store['date'].isin(high_outliers['date']))]
print(f"Normal 2015 Saturday avg: {normal_saturdays_2015['revenue'].mean()}")

Save in drive

In [ ]:
# --- Save Processed DataFrames ---

PROJECT_PATH = '/content/drive/MyDrive/M5_Project/'
output_folder = PROJECT_PATH + 'processed_data/'

# 1. Save the main featured dataframe
df_featured.to_parquet(output_folder + 'df_featured.parquet', engine='pyarrow')

# 2. Save the original merged dataframe
original_df.to_parquet(output_folder + 'original_df.parquet', engine='pyarrow')

print(f"Successfully saved files to: {output_folder}")
print(os.listdir(output_folder)) # List files to confirm they were saved

Split train/test

In [ ]:
# --- train and validation sets from the FEATURED data ---
last_train_date_featured = df_featured['date'].max() - pd.to_timedelta(H, unit='D')
validation_start_date_featured = last_train_date_featured + pd.to_timedelta(1, unit='D')

train_final = df_featured[df_featured['date'] <= last_train_date_featured]
val_final = df_featured[df_featured['date'] >= validation_start_date_featured]

# --- Define our features (X) and target (y) ---
TARGET = 'revenue'

FEATURES = [col for col in train_final.columns if col not in [TARGET, 'store_name', 'date', 'weekday', 'event']]

X_train = train_final[FEATURES]
y_train = train_final[TARGET]

X_val = val_final[FEATURES]
y_val = val_final[TARGET]

print("Training data shape (X, y):", X_train.shape, y_train.shape)
print("Validation data shape (X, y):", X_val.shape, y_val.shape)
print("\nHere are a few of the features we created:")
print(X_train.head())

In [ ]:
original_df.columns